In [78]:
!pip install gdown
!pip install requests

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gdown: filename=gdown-4.5.1-py3-none-any.whl size=14951 sha256=64d8e41ce87c0cf57c3a58ea11aef8468636c5c95d59f41eb5de2092827de899
  Stored in directory: /tmp/pip-ephem-wheel-cache-trs32hce/wheels/9b/0f/b5/18491a0b635e27a4ca24e38909932443118ce3874d682514a8
Successfully built gdown


In [12]:
import gdown

google_path = 'https://drive.google.com/uc?id='
file_id = '1--uZb0fSu-S7PDaMVnxbKnBaClR2phWv'
output_name = 'total_2020.csv'
gdown.download(google_path+file_id,output_name,quiet=False)

file_id = '1AE8lc6S-RFjcS7rNdRo6xIN6YsQH69QI'
output_name = 'total_2019.csv'
gdown.download(google_path+file_id,output_name,quiet=False)

file_id = '1s6U9VlgeBvERVFcgJvjUjFMWqcNzHq3q'
output_name = 'total_2021.csv'
gdown.download(google_path+file_id,output_name,quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1--uZb0fSu-S7PDaMVnxbKnBaClR2phWv
To: /opt/app-root/src/지수 및 모델링/total_2020.csv
100%|██████████| 235k/235k [00:00<00:00, 1.50MB/s]
Downloading...
From: https://drive.google.com/uc?id=1AE8lc6S-RFjcS7rNdRo6xIN6YsQH69QI
To: /opt/app-root/src/지수 및 모델링/total_2019.csv
100%|██████████| 235k/235k [00:00<00:00, 1.52MB/s]
Downloading...
From: https://drive.google.com/uc?id=1s6U9VlgeBvERVFcgJvjUjFMWqcNzHq3q
To: /opt/app-root/src/지수 및 모델링/total_2021.csv
100%|██████████| 42.4k/42.4k [00:00<00:00, 951kB/s]


'total_2021.csv'

In [41]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

import requests

df_2020 = pd.read_csv("total_2020.csv")
df_2019 = pd.read_csv("total_2019.csv")
df_2021 = pd.read_csv("total_2021.csv")
grid = gpd.read_file("../Data/5.김해시_격자(100X100).geojson")

df = pd.concat([df_2019, df_2020]).reset_index(drop=True)

df = df[['대지위치','대지면적(㎡)', '신고일']]

In [42]:
df = pd.concat([df, df_2021[['대지위치','대지면적(㎡)', '신고일']]]).reset_index(drop=True)
df

,대지위치,대지면적(㎡),신고일
0,경상남도 김해시 신문동 504-14 외1필지,230,2019-01-21
1,경상남도 김해시 유하동 255-1,783,2019-01-21
2,경상남도 김해시 화목동 1095,256,2019-01-18
3,경상남도 김해시 상동면 우계리 134-2,"2,243",2019-01-09
4,경상남도 김해시 지내동 193-16,597.7,2019-01-07
...,...,...,...
1450,경상남도 김해시 생림면 나전리 43-2 외21필지,"6,398",2021-06-30
1451,경상남도 김해시 내동 1116-5,"1,742.9",2020-07-22
1452,경상남도 김해시 안동 583-8 외2필지,"1,349",2019-12-26
1453,경상남도 김해시 주촌면 양동리 352-2 외3필지,846,2019-12-23


In [43]:
# 사용예제 http://api.vworld.kr/req/address?service=address&request=getcoord&version=2.0&crs=epsg:4326&address=%ED%9A%A8%EB%A0%B9%EB%A1%9C72%EA%B8%B8%2060&refine=true&simple=false&format=json&type=road&key=[KEY]

url_front = "http://api.vworld.kr/req/address?"
url_params = "service=address&request=getcoord&version=2.0&crs=epsg:4326&refine=true&simple=false&format=json&type=PARCEL"
url_address = "&address="
url_key = "&key="

#개인 API 없으면 제꺼 쓰고 있으면 각자 꺼! 하루에 몇 건 제한있다고 해성
auth_key = "DA366A28-291B-3142-9D13-876C510A16F9" 

points = []
addressList = (df['대지위치']).values

for i in addressList:
    url = url_front + url_params + url_address + i + url_key + auth_key

    result = requests.get(url)
    json_data = result.json()

    if json_data['response']['status'] == 'OK':
        x = float(json_data['response']['result']['point']['x'])
        y = float(json_data['response']['result']['point']['y'])
        points.append(Point(x,y))
    else:
        points.append(-1)

In [44]:
df['point'] = points
df

,대지위치,대지면적(㎡),신고일,point
0,경상남도 김해시 신문동 504-14 외1필지,230,2019-01-21,POINT (128.814655051964 35.1960831020379)
1,경상남도 김해시 유하동 255-1,783,2019-01-21,POINT (128.819985821133 35.2136076162417)
2,경상남도 김해시 화목동 1095,256,2019-01-18,POINT (128.851479992388 35.206889921858)
3,경상남도 김해시 상동면 우계리 134-2,"2,243",2019-01-09,POINT (128.912430313177 35.3025517472532)
4,경상남도 김해시 지내동 193-16,597.7,2019-01-07,POINT (128.920575328343 35.227485194875)
...,...,...,...,...
1450,경상남도 김해시 생림면 나전리 43-2 외21필지,"6,398",2021-06-30,-1
1451,경상남도 김해시 내동 1116-5,"1,742.9",2020-07-22,POINT (128.865102205618 35.2350879422505)
1452,경상남도 김해시 안동 583-8 외2필지,"1,349",2019-12-26,POINT (128.913675227867 35.2250745652042)
1453,경상남도 김해시 주촌면 양동리 352-2 외3필지,846,2019-12-23,POINT (128.800347552918 35.2282787308837)


In [46]:
gid = []
check = []
for index, row in df.iterrows():
    flag = 0
    if row['point']==-1:
        flag = 1
        gid.append(-1)
    else:
        for INDEX, ROW in grid.iterrows():
            if row['point'].within(ROW['geometry']):
                gid.append(ROW['gid'])
                flag = 1
                break
    if flag == 0:
        print('얘는 gid가 없는데?')
        gid.append(-1)
        print(index)
    if index%100 == 0:
        print(index)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400


In [48]:
df['gid'] = gid
df_train = df[:1013]
df_test = df[490:]
df_train.to_csv("building_train.csv", index=False)
df_test.to_csv("building_test.csv", index=False)

In [51]:
df_test

,대지위치,대지면적(㎡),신고일,point,gid
490,경상남도 김해시 장유동 819-11,264.2,2020-02-27,POINT (128.824494966076 35.1734656291788),마라206872
491,경상남도 김해시 진영읍 죽곡리 4-1,"4,071.4",2020-02-27,POINT (128.769963105048 35.2892457016956),마마154000
492,경상남도 김해시 진례면 고모리 산 19 외4필지,"21,468",2020-02-25,POINT (128.778267026835 35.2816753325834),마라162992
493,경상남도 김해시 주촌면 천곡리 871-1,262,2020-02-25,POINT (128.828625217528 35.2333724938705),마라208939
494,경상남도 김해시 대동면 주동리 446-13,264,2020-02-24,POINT (128.946902659399 35.2384882405878),마라316946
...,...,...,...,...,...
1450,경상남도 김해시 생림면 나전리 43-2 외21필지,"6,398",2021-06-30,-1,-1
1451,경상남도 김해시 내동 1116-5,"1,742.9",2020-07-22,POINT (128.865102205618 35.2350879422505),마라242941
1452,경상남도 김해시 안동 583-8 외2필지,"1,349",2019-12-26,POINT (128.913675227867 35.2250745652042),마라286931
1453,경상남도 김해시 주촌면 양동리 352-2 외3필지,846,2019-12-23,POINT (128.800347552918 35.2282787308837),마라183933
